In [ ]:
from pyspark.sql import SparkSession


URL_REPRES = "jdbc:sqlserver://Admin\TRAM1:1433;databaseName=RepresentativeOffice;user=sa;password=12345;"
URL_SALE = "jdbc:sqlserver://Admin\TRAM1:1433;databaseName=Sale;user=sa;password=12345;"
URL_IDB =  "jdbc:sqlserver://Admin\TRAM1:1433;databaseName=Integration;user=sa;password=12345;"
spark = SparkSession.builder \
    .appName("Read from MSSQL") \
    .config("spark.driver.extraClassPath", "C:/SPARK/spark-3.5.1-bin-hadoop3/jars/apache-spark-sql-connector.jar") \
    .getOrCreate()
url = "jdbc:sqlserver://Admin\TRAM1;databaseName=RepresentativeOffice;user=sa;password=12345;"
properties = {
    "user": "sa",
    "password": "12345",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

In [ ]:
m1 = '2008-01-01'
m2 = '2023-01-01'
m3 = '2023-12-02'

In [ ]:
def sale_to_idb(sale_table,idb_table,start, end):
    condition = f"time<='{end}' and time >'{start}'"
    if sale_table=="Orders":
        condition = f"OrderDate<='{end}' and OrderDate >'{start}'"
    print(sale_table,condition)
    df = spark.read.jdbc(url=URL_SALE, table=sale_table, properties=properties).filter(condition)
    df.write.jdbc(url=URL_IDB,
              table=idb_table,
              mode = "append",
              properties=properties)
    return 0

In [ ]:
def repres_to_idb(repres_table,idb_table,start, end):
    condition = f"time<='{end}' and time >'{start}'"
    if repres_table=="Customers":
        condition = f"FirstOrderDate<='{end}' and FirstOrderDate >'{start}'"
    print(repres_table,condition)
    df = spark.read.jdbc(url=URL_REPRES, table=repres_table, properties=properties).filter(condition)
    df.write.jdbc(url=URL_IDB,
              table=idb_table,
              mode = "append",
              properties=properties)
    return 0

In [ ]:
move_ls = [("RepresentativeOffices","RepresentativeOffices","sale"),
           ("Stores","Stores","sale"),
           ("Items","Items","sale"),
           ("Customers","Customers","repres"),
           ("TravelCustomers","TravelCustomers","repres"),
           ("PostalCustomers","PostalCustomers","repres"),
           ("StoredItems","StoredItems","sale"),
           ("Orders","Orders","sale"),
           ("OrderedItems","OrderedItems","sale")
          ]
           

In [ ]:
for x in move_ls:
    if x[2]=="sale":
        sale_to_idb(x[0],x[1],m1,m2)
    else:
        repres_to_idb(x[0],x[1],m1,m2)

In [ ]:
for x in move_ls:
    if x[2]=="sale":
        sale_to_idb(x[0],x[1],m2,m3)
    else:
        repres_to_idb(x[0],x[1],m2,m3)